# How to use OpenCV to segment an image with the region growing algorithm

## Pseudo code

```cpp
seed_point // coordinates of the starting point
visited_matrix // boolean array/matrix, same size as image
               // all the pixels are initialised to false
point_list // empty list

point_list.enqueue( seed_point ) // Add the seed to the list

while ( point_list is not empty )
{
    this_point = point_list.dequeue() // Get a point from the list
    visited_matrix( this_point ) = true // Visit the point

    for each neighbour of this_point
    {
        if not visited_matrix( neighbour ) and neighbour is similar to this_point and neighbour not in point_list
        {
            point_list.enqueue( neighbour )
        }
    }
}
```

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <vector>
#include <cmath>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Read the image from a file on the disk and return a new matrix

![CT slice](../CT.png)

In [5]:
Mat ct_slice = imread("../CT.png", IMREAD_GRAYSCALE);

## Check for errors

In [6]:
// Check for failure
if (ct_slice.empty()) 
{
    cerr << "Could not open or find the image" << endl;
    return -1;
}

## Display the image and pickup seeds

In [7]:
vector<pair<int, int>> seed_set; // Stores the seeds

In [8]:
void mouseCallback(int event, int x, int y, int flags, void* userdata)
{
    if  ( event == EVENT_LBUTTONDOWN )
    {
        seed_set.push_back(pair<int, int>(x, y));
        Mat* p_image = static_cast<Mat*>(userdata);
        cv::Scalar colour( 0, 0, 255 );
        cv::circle( *p_image, Point(x, y), 4, colour, FILLED );
        imshow("CT slice", *p_image);
    }
}

In [9]:
Mat colour_ct_slice;
cvtColor(ct_slice, colour_ct_slice, COLOR_GRAY2RGB);

namedWindow("CT slice", WINDOW_AUTOSIZE); // Create a window
imshow("CT slice", colour_ct_slice); // Show our image inside the created window
setMouseCallback("CT slice", mouseCallback, &colour_ct_slice); // Register the callback function
waitKey(0); // Wait for any keystroke in the window

113

Here we selected 3 seedsto segment the lungs

![CT slice and the 3 seeds](../opencv-cxx-region_growing1.png)

## Region growing function

In [10]:
Mat regionGrowing(const Mat& anImage,
                  const vector<pair<int, int>>& aSeedSet,
                  unsigned char anInValue = 255,
                  float tolerance = 5)
{
    // boolean array/matrix, same size as image
    // all the pixels are initialised to false
    Mat visited_matrix = Mat::zeros(Size(anImage.cols, anImage.rows), CV_8UC1);

    // List of points to visit
    vector<pair<int, int>> point_list = aSeedSet;

    while ( ! point_list.empty() )
    {
        // Get a point from the list
        pair<int, int> this_point = point_list.back();
        point_list.pop_back();
        
        int x = this_point.first;
        int y = this_point.second;
        unsigned char pixel_value = anImage.at<unsigned char>(Point(x,y));
                                                                                                                            
        // Visit the point
        visited_matrix.at<unsigned char>(Point(x, y)) = anInValue;

        // for each neighbour of this_point
        for (int j = y - 1; j <= y + 1; ++j)
        {
            // vertical index is valid
            if (0 <= j && j < anImage.rows)
            {
                for (int i = x - 1; i <= x + 1; ++i)
                {
                    // hozirontal index is valid
                    if (0 <= i && i < anImage.cols)
                    {
                        unsigned char neighbour_value = anImage.at<unsigned char>(Point(i, j));
                        unsigned char neighbour_visited = visited_matrix.at<unsigned char>(Point(i, j));
                        
                        if (!neighbour_visited &&
                            fabs(neighbour_value - pixel_value) <= (tolerance / 100.0 * 255.0)) // neighbour is similar to this_point
                        {
                            point_list.push_back(pair<int, int>(i, j));
                        }
                    }
                }
            }
        }
    }

    return visited_matrix;
}

## Segment the input

In [11]:
Mat segmented_image = regionGrowing(ct_slice, seed_set, 255, 2);

## Display the result

In [12]:
namedWindow("Segmentation", WINDOW_AUTOSIZE); // Create a window
imshow("Segmentation", segmented_image); // Show our image inside the created window

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy the created windows

Here is the result

![Segmentation](../opencv-cxx-region_growing2.png)